# Q5) Predict the sentiment of tweets by followers of naveen_odisha

In [ ]:
import TwitterSearch
from TwitterSearch import *

try:
    tuo = TwitterUserOrder('naveen_odisha') # create a TwitterUserOrder

    # it's about time to create TwitterSearch object again
    ts = TwitterSearch(
        consumer_key = "LsEFjnM2sH6HJmXMfSj0z21OH",
        consumer_secret = "HOnfLgJDfnA1m2dyem1THUorpYFsHE2BGzS6IFenbVboHKFArE",
        access_token = "4445875228-39Rv7h0FvDbv3XuMpiQliO3HcO2HrL2ZokovxPi",
        access_token_secret = "RMdIGwd4CZams3CZsflh2DOpk6ACoxA1UFB70KH47fzZG"
    )

    # start asking Twitter about the timeline
    for tweet in ts.search_tweets_iterable(tuo):
        print( '@%s tweeted: %s' % ( tweet['user']['screen_name'], tweet['text'] ) )

except TwitterSearchException as e: # catch all those ugly errors
    print(e)

In [ ]:
import json
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
from textblob import TextBlob
from elasticsearch import Elasticsearch

# import twitter keys and tokens
from config import *

# create instance of elasticsearch
es = Elasticsearch()


class TweetStreamListener(StreamListener):

    # on success
    def on_data(self, data):

        # decode json
        dict_data = json.loads(data)

        # pass tweet into TextBlob
        tweet = TextBlob(dict_data["text"])

        # output sentiment polarity
        print tweet.sentiment.polarity

        # determine if sentiment is positive, negative, or neutral
        if tweet.sentiment.polarity < 0:
            sentiment = "negative"
        elif tweet.sentiment.polarity == 0:
            sentiment = "neutral"
        else:
            sentiment = "positive"

        # output sentiment
        print sentiment

        # add text and sentiment info to elasticsearch
        es.index(index="sentiment",
                 doc_type="test-type",
                 body={"author": dict_data["user"]["screen_name"],
                       "date": dict_data["created_at"],
                       "message": dict_data["text"],
                       "polarity": tweet.sentiment.polarity,
                       "subjectivity": tweet.sentiment.subjectivity,
                       "sentiment": sentiment})
        return True

    # on failure
    def on_error(self, status):
        print status

if __name__ == '__main__':

    # create instance of the tweepy tweet stream listener
    listener = TweetStreamListener()

    # set twitter keys/tokens
    auth = OAuthHandler("LsEFjnM2sH6HJmXMfSj0z21OH", "HOnfLgJDfnA1m2dyem1THUorpYFsHE2BGzS6IFenbVboHKFArE")
    auth.set_access_token("4445875228-39Rv7h0FvDbv3XuMpiQliO3HcO2HrL2ZokovxPi","RMdIGwd4CZams3CZsflh2DOpk6ACoxA1UFB70KH47fzZG")

    # create instance of the tweepy stream
    stream = Stream(auth, listener)

    # search twitter for "congress" keyword
    stream.filter(track=['naveen_odisha'])

401
401
401
401
401
401
401
401
401
401

In [ ]:
import time
from tweepy import Stream
from tweepy import OAuthHandler
from tweepy.streaming import StreamListener
import os